In [42]:
import random
import math
import copy

from collections import defaultdict
def isSorted(nodes):
    if(len(nodes)==0):
        return True
    for i in range(1,len(nodes)):
        if(nodes[i-1] > nodes[i]):
            return False
    return True

def getEuclideanDistanceBetweenNodes(node, otherNode, AdjHash, Map):
    if(node == otherNode):
        return 0;
    if(otherNode in AdjHash and node in AdjHash[otherNode]):
        return AdjHash[otherNode][node]
    return EuclideanDistance(Map[node][0], Map[node][1], Map[otherNode][0], Map[otherNode][1])

def computeAdjHashOfCompleteGraph(Map):
    # Input::  Map: {0: (0.13037375991631983, 0.17982980099790336), ... }
    nodes = Map.keys()
    assert isSorted(nodes)
    AdjHash= {}
    for node in nodes:
        AdjHash[node] = {}
        for otherNode in nodes:
            AdjHash[node][otherNode] = getEuclideanDistanceBetweenNodes(node, otherNode, AdjHash, Map)
    return AdjHash

def TspGenerator(numberOfCities, lowRange=0.0, highRange=1.0):
    Map = {}
    inverseMap = {}
    AdjHash = {}
    for x in range(0, numberOfCities):
        coordinate = (random.uniform(lowRange, highRange), random.uniform(lowRange, highRange))
        tries = 0;
        while(coordinate in inverseMap):
            coordinate = (random.uniform(lowRange, highRange), random.uniform(lowRange, highRange))
            tries+=1;
            if(tries==5):
                print "Unable to Generate Coordinates"
                return ;
        Map[x] = coordinate
        inverseMap[coordinate] = x
    return Map, inverseMap, computeAdjHashOfCompleteGraph(Map);

def EuclideanDistance(point1_x, point1_y, point2_x, point2_y):
    internalCalculation = math.pow((point1_x - point2_x),2) + math.pow((point1_y - point2_y),2)
    return math.sqrt(internalCalculation);


In [109]:
def isAllVisited(VisitedHash, nodes):
    for node in nodes:
        if(node not in VisitedHash):
            return False
    return True

def PrimsAlgorithm(startNode, nodes, AdjHash):
    #Input startNode : type int, nodes : type list of int's
    MSTCost = 0;
    h = []
    visitedHash = {}
    visitedHash[startNode] = True
    prevNode = startNode
    MstEdges = []
    while(not isAllVisited(visitedHash, nodes)):
        for node in nodes:
            if(node not in visitedHash.keys()):
                heappush(h, (AdjHash[prevNode][node], (prevNode, node)))
        cost, edge = heappop(h)
        parentNode, minNode = edge
        while(minNode in visitedHash):
            # if min node was alreadyVisited
            cost, edge = heappop(h)
            parentNode, minNode = edge
        MSTCost += cost
        MstEdges.append((parentNode, minNode))
        visitedHash[minNode] = True
    return MSTCost, MstEdges;

def MSTHeuristic(startNode, nodes, AdjHash):
    #Input :: AdjHash is a Adjacency map of the entire set of nodes with the value being the distance
    assert startNode in nodes;
    PrimsAlgorithm(startNode, nodes, AdjHash)
    
    
class TspNode:
    def __init__(self, x, y, nodeNumber, path_cost, heuristicCost, parentNode):
        self.pos_x = x
        self.pos_y = y
        self.node_number = nodeNumber
        self.h_cost = heuristicCost
        self.g_cost = path_cost
        self.f_cost = self.g_cost + self.h_cost
        self.parent = parentNode

def goalTest(node, listOfNodeNumbers):
    return isHamiltonianCycle(node, listOfNodeNumbers)

def appendToFrontier(newNode, Frontier):
    if(newNode.f_cost in Frontier):
        Frontier[newNode.f_cost].append(newNode)
    else:
        Frontier[newNode.f_cost] = [newNode]

def printPathTraversed(node):
    pathTraversed = []
    while(node.parent!=None):
        pathTraversed.append(node.node_number)
        node=node.parent
    pathTraversed.append(node.node_number)
    return pathTraversed
    
def isHamiltonianCycle(node, listOfNodeNumbers):
    pathTraversed = [node.node_number]
    visited = {}
    while(node.parent!=None):
        pathTraversed.append(node.node_number)
        visited[node.node_number] = True
        node = node.parent
    pathTraversed.append(node.node_number)
    visited[node.node_number] = True
    # Check if ALL NODES are visited
    for nodeNumber in listOfNodeNumbers:
        if nodeNumber not in visited:
            return False
    # Check if cycle is present
    if(pathTraversed[0] == pathTraversed[len(pathTraversed)-1]):
        return True
    return False;

def popMinNode(Frontier):
    m = min(i for i in Frontier.keys() if len(Frontier[i]) > 0)
    minNode = Frontier[m].pop(0)
    return minNode;
    
def findBestNode(Frontier):
    bestNode = popMinNode(Frontier)
    return bestNode
        
def checkIfFCostIsGreaterThanBestFinalNode(bestFinalNode, node):
    if(bestFinalNode.f_cost <= node.f_cost):
        return False
    return True

def checkNoOtherBetterNodeToExpand(Frontier, finalNode):
    bestNodeToExpand = findBestNode(Frontier)
    return checkIfFCostIsGreaterThanBestFinalNode(finalNode, bestNodeToExpand)

def appendToFrontier(newNode, Frontier):
    if(newNode.f_cost in Frontier):
        Frontier[newNode.f_cost].append(newNode)
    else:
        Frontier[newNode.f_cost] = [newNode]

def expandNode(Node, Frontier, AdjHash, listOfNodeNumbers, Map, heuristicCost):
    #TO-DO add optimization for not adding an edge already traversed.
    for nodeNumber in listOfNodeNumbers:
        if(nodeNumber != Node.node_number):
            latitude = Map[nodeNumber][0]
            longitude = Map[nodeNumber][1]
            tspNode = TspNode(latitude, longitude, nodeNumber, Node.g_cost+AdjHash[Node.node_number][nodeNumber], heuristicCost, Node)
            appendToFrontier(tspNode, Frontier)
    return ;
    
def solveAstar(startNode, AdjHash, listOfNodeNumbers, Map, heuristicCost):
    #Input startNode : type int, listOfNodeNumbers : type list of int's
    latitude = Map[startNode][0]
    longitude = Map[startNode][1]
    sNode = TspNode(latitude, longitude, startNode, 0, heuristicCost, None)
    Frontier = defaultdict(list)
    Frontier[sNode.f_cost] = [sNode]
    isCompleted = False
    bestFinalNode = None
    FoundCostToFinal = False;
    noOfNodesExpanded = 0
    while(len(Frontier)!=0 or isCompleted):
        bestNode = findBestNode(Frontier)
        if(bestNode.g_cost < heuristicCost):
            isNodeGoalState = goalTest(bestNode, listOfNodeNumbers)
            if(FoundCostToFinal):
                if(not checkIfFCostIsGreaterThanBestFinalNode(bestFinalNode, bestNode)):
                    break
            if(isNodeGoalState):
                if(not FoundCostToFinal):
                    bestFinalNode = bestNode
                    FoundCostToFinal = True
                else:
                    if(bestFinalNode.g_cost > bestNode.g_cost):
                        bestFinalNode=bestNode
            if(isNodeGoalState and checkNoOtherBetterNodeToExpand(Frontier, bestNode)):
                isCompleted = True
                break
            print "Expanding Node Number %d with co-ordinates (%s,%s) " %(bestNode.node_number, bestNode.pos_x, bestNode.pos_y)
            noOfNodesExpanded+=1
            expandNode(bestNode, Frontier, AdjHash, listOfNodeNumbers, Map, heuristicCost)
        else:
            break
    return bestFinalNode, noOfNodesExpanded

In [105]:
Map, inverseMap, AdjHash = TspGenerator(5)
nodes = Map.keys()

In [106]:
heuristicCost, EdgesMst = PrimsAlgorithm(0, nodes, AdjHash)
heuristicCost *= 2;

In [110]:
startNode = 0
bestFinalNode, noOfNodesExpanded = solveAstar(startNode, AdjHash, nodes, Map, heuristicCost)

Expanding Node Number 0 with co-ordinates (0.713931847749,0.260390064505) 
Expanding Node Number 3 with co-ordinates (0.559234371158,0.403924489292) 
Expanding Node Number 1 with co-ordinates (0.61556511328,0.584583872375) 
Expanding Node Number 4 with co-ordinates (0.953043721349,0.559747507829) 
Expanding Node Number 1 with co-ordinates (0.61556511328,0.584583872375) 
Expanding Node Number 0 with co-ordinates (0.713931847749,0.260390064505) 
Expanding Node Number 3 with co-ordinates (0.559234371158,0.403924489292) 
Expanding Node Number 3 with co-ordinates (0.559234371158,0.403924489292) 
Expanding Node Number 2 with co-ordinates (0.181755697045,0.565923190988) 
Expanding Node Number 2 with co-ordinates (0.181755697045,0.565923190988) 
Expanding Node Number 3 with co-ordinates (0.559234371158,0.403924489292) 
Expanding Node Number 4 with co-ordinates (0.953043721349,0.559747507829) 
Expanding Node Number 4 with co-ordinates (0.953043721349,0.559747507829) 
Expanding Node Number 0 wit

In [111]:
printPathTraversed(bestFinalNode)

[0, 4, 1, 2, 3, 0]

In [112]:
# Testing Prims Algorithm
ExamplestartNode = 1
Examplenodes = [1,2,3,4]
ExampleAdjHash =  {
    1:{ 
        1: 0,
        2: 10,
        3: 15,
        4: 20
      },
    2:{ 
        1: 10,
        2: 0,
        3: 35,
        4: 25
      },
    3:{ 
        1: 15,
        2: 35,
        3: 0,
        4: 30
      },
    4:{ 
        1: 20,
        2: 25,
        3: 30,
        4: 0
      }
}
ExampleMap = {
    1:(0,0),
    2:(1,1),
    3:(2,2),
    4:(3,3)
}
ExampleheuristicCost, Edges = PrimsAlgorithm(ExamplestartNode, Examplenodes, ExampleAdjHash)
ExampleheuristicCost *= 2

In [113]:
ExampleheuristicCost

90

In [116]:
bestnode, noOfNodesExpanded = solveAstar(ExamplestartNode, ExampleAdjHash, Examplenodes, ExampleMap, ExampleheuristicCost)

Expanding Node Number 1 with co-ordinates (0,0) 
Expanding Node Number 2 with co-ordinates (1,1) 
Expanding Node Number 3 with co-ordinates (2,2) 
Expanding Node Number 4 with co-ordinates (3,3) 
Expanding Node Number 1 with co-ordinates (0,0) 
Expanding Node Number 1 with co-ordinates (0,0) 
Expanding Node Number 2 with co-ordinates (1,1) 
Expanding Node Number 4 with co-ordinates (3,3) 
Expanding Node Number 3 with co-ordinates (2,2) 
Expanding Node Number 1 with co-ordinates (0,0) 
Expanding Node Number 4 with co-ordinates (3,3) 
Expanding Node Number 2 with co-ordinates (1,1) 
Expanding Node Number 1 with co-ordinates (0,0) 
Expanding Node Number 3 with co-ordinates (2,2) 
Expanding Node Number 4 with co-ordinates (3,3) 
Expanding Node Number 2 with co-ordinates (1,1) 
Expanding Node Number 3 with co-ordinates (2,2) 
Expanding Node Number 2 with co-ordinates (1,1) 
Expanding Node Number 3 with co-ordinates (2,2) 
Expanding Node Number 4 with co-ordinates (3,3) 
Expanding Node Numbe

In [120]:
print printPathTraversed(bestnode)
print noOfNodesExpanded

[1, 3, 4, 2, 1]
109
